In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-02-25"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13183,2024-02-25,Grécia A1 Feminina,10:00,Olympiakos F,Panathinaikos F,1.43,2.68,139.5,1.80,1.86,0.0,0.00,0.00,QXCxgKea,0.699301,0.373134,0.555556,0.537634,0.072435,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,91,90,28.00,1.04,0.000,0.000,0.430114,0.023184,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13184,2024-02-25,Grécia Liga De Elite,10:00,Milon Aons,Eleftheroupoli,1.51,2.45,153.5,1.88,1.79,0.0,0.00,0.00,n9YMpmpd,0.662252,0.408163,0.531915,0.558659,0.070415,128.970,17.549373,0.136073,1.8,1.643168,0.912871,109.48,1.624,0.236707,0.145755,3.0,122.474,38.067273,0.310819,1.8,1.643168,0.912871,81.74,1.604,0.387724,0.241723,12.0,68,67,1.61,1.22,116.432,155.104,0.335697,0.034681,NaN,-0.63,-0.126,-4.047619,0.000000,0.0,0.000000,3.52,0.704,2.059659,0.000000,0.0,0.000000
13185,2024-02-25,Grécia A1 Feminina,08:00,GAS Evnikos F,Esperides Kalliotheas F,2.72,1.42,137.5,1.88,1.79,0.0,0.00,0.00,dIlFyJQO,0.367647,0.704225,0.531915,0.558659,0.071872,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,228.174,280.208115,1.228046,1.8,1.643168,0.912871,729.30,2.966,3.571978,1.204308,57.0,66,78,3.15,9.35,0.000,0.000,0.444077,0.034681,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13186,2024-02-25,Grécia A1 Feminina,09:00,Athinaikos F,PAOK F,6.04,1.10,143.5,1.82,1.84,0.0,0.00,0.00,8dIXg0tg,0.165563,0.909091,0.549451,0.543478,0.074654,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,134.644,34.823631,0.258635,1.8,1.643168,0.912871,132.86,2.000,0.524404,0.262202,-7.0,51,73,13.10,1.82,0.000,0.000,0.978461,0.007728,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13187,2024-02-25,Grécia A1 Feminina,09:00,Eleftheria F,Proteas Voulas F,1.56,2.32,132.5,1.85,1.85,0.0,0.00,0.00,bNSc023P,0.641026,0.431034,0.540541,0.540541,0.072060,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,255.066,251.914390,0.987644,1.2,1.643168,1.369306,79.79,4.226,4.700078,1.112181,1.0,78,79,1.30,1.01,0.000,0.000,0.277011,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13319,2024-02-25,Eua Ncaa,01:59,Hawaii,Long Beach State,1.67,2.28,150.5,1.80,1.95,-4.5,2.05,3.15,rBdAvbOe,0.598802,0.438596,0.555556,0.512821,0.037399,169.084,63.119712,0.373304,1.8,1.643168,0.912871,142.00,2.180,1.040697,0.477384,23.0,133.000,42.573235,0.320100,3.0,0.000000,0.000000,91.64,1.630,0.498949,0.306104,49.0,71,79,2.00,1.16,118.930,145.224,0.218398,0.056569,0.299161,1.03,0.206,3.252427,0.651030,0.5,-0.151030,0.79,0.158,8.101266,0.519155,0.7,0.180845
13320,2024-02-25,Eua Ncaa,14:00,La Salle,Rhode Island,1.69,2.18,153.5,1.83,1.83,0.0,0.00,0.00,h4b7GvJ9,0.591716,0.458716,0.546448,0.546448,0.050432,235.836,82.137909,0.348284,0.6,1.341641,2.236068,245.18,3.352,1.510354,0.450583,-55.0,219.202,102.378704,0.467052,1.2,1.643168,1.3693

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
13201,01:00,Japão B2.League,Yamagata Wyverns,Fukushima,1.99,Back Home
13220,13:00,Turquia Tb2L,Cozum Ankara,Karamursel,2.05,Back Home
13223,08:00,Turquia Tkbl Feminina,Emlak Konut Gelisim 2 F,Burhaniye F,1.62,Back Home
13250,08:00,Portugal Lfb Feminina,Galitos F,Natacao F,1.89,Back Home
13275,19:00,Eua Nba,Indiana Pacers,Dallas Mavericks,1.88,Back Home
13279,21:00,Eua Nba,Golden State Warriors,Denver Nuggets,1.98,Back Home
13314,14:00,Eua Ncaa,St. Johns,Creighton,2.10,Back Home
